In [1]:
import sys
import random
import numpy as np
import os
from PIL import Image
from src.env.env import RILAB_OMY_ENV
import gc, argparse
import json
import copy
import time
from src.controllers import load_controller
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset
import glfw

/home/rilab/ros2_ws/src/deep_learning_proj/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment configuration
config_file_path = './configs/train_key.json'
with open(config_file_path) as f:
    env_conf = json.load(f)
language_instruction = env_conf['language_instruction']
omy_env = RILAB_OMY_ENV(cfg=env_conf, seed=None, 
                        action_type=env_conf['control_mode'], 
                        obs_type='full_state',
                        vis_mode = 'keyboard')


omy_env.reset(leader_pose = False)
# Load keyboard controller
controller = load_controller('keyboard',env_conf)
controller.reset(omy_env)


-----------------------------------------------------------------------------
name:[tabletop_env] dt:[0.002] HZ:[500]
 n_qpos:[34] n_qvel:[31] n_qacc:[31] n_ctrl:[8]
 integrator:[IMPLICITFAST]

n_body:[25]
 [0/25] [world] mass:[0.00]kg
 [1/25] [front_object_table] mass:[1.00]kg
 [2/25] [camera] mass:[0.00]kg
 [3/25] [camera2] mass:[0.00]kg
 [4/25] [camera3] mass:[0.00]kg
 [5/25] [base] mass:[3.59]kg
 [6/25] [link1] mass:[2.06]kg
 [7/25] [link2] mass:[3.68]kg
 [8/25] [link3] mass:[2.39]kg
 [9/25] [link4] mass:[1.40]kg
 [10/25] [link5] mass:[1.40]kg
 [11/25] [link6] mass:[0.65]kg
 [12/25] [camera_center] mass:[0.00]kg
 [13/25] [tcp_link] mass:[0.32]kg
 [14/25] [rh_p12_rn_r1] mass:[0.07]kg
 [15/25] [rh_p12_rn_r2] mass:[0.02]kg
 [16/25] [rh_p12_rn_l1] mass:[0.07]kg
 [17/25] [rh_p12_rn_l2] mass:[0.02]kg
 [18/25] [body_obj_wooden_cabinet] mass:[2.24]kg
 [19/25] [body_obj_wooden_cabinet_top] mass:[3.00]kg
 [20/25] [body_obj_wooden_cabinet_middle] mass:[3.00]kg
 [21/25] [body_obj_wooden_cabin

You can teleop your robot with keyboard
```
---------     -----------------------
   w       ->        backward
s  a  d        left   forward   right
---------      -----------------------
In x, y plane

---------
R: Moving Up
F: Moving Down
---------
In z axis

---------
Q: Tilt left
E: Tilt right
UP: Look Upward
Down: Look Donward
Right: Turn right
Left: Turn left
---------
For rotation

---------
SPACEBAR: Toggle Gripper
--------

---------
z: reset
--------
```

In [ ]:
while omy_env.env.is_viewer_alive():
    omy_env.step_env()
    if omy_env.env.loop_every(HZ=20):
        temp = omy_env.get_object_pose(pad=5)
        delta_eef = controller.get_action()
        omy_env.step(delta_eef)
        omy_env.grab_image()
        omy_env.render(task=language_instruction)
        success = omy_env.check_success()
        if omy_env.env.is_key_pressed_once(glfw.KEY_Z):
            omy_env.reset(leader_pose = False)
        if success or omy_env.env.is_key_pressed_once(glfw.KEY_ESCAPE):
            break
    omy_env.env.sync_sim_wall_time()
omy_env.env.close_viewer()
print("Episode ended.")
print(f"Success: {success}")

Episode ended.
Success: True
